In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('./YoutubeCommentsDataSet.csv')
df.head()

In [ ]:
df.duplicated().sum()

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

In [ ]:
print(f"Percent of values to drop in the dataset: {round((44+531) / len(df), 3) * 100}%")

Since the missing values are just less than 4% of the original dataset, we can just drop it

In [ ]:
sns.set_theme(style='darkgrid')
sns.countplot(data = df, x = 'Sentiment', hue='Sentiment', palette='magma')
plt.ylabel("Count")
plt.show()

In [ ]:
df['Sentiment'].replace({'positive': 2, 
                         'negative': 0,
                         'neutral': 1}, inplace=True)

df.head()

# MODEL TIME!!!

In [ ]:
# Import necessary libraries
from sklearn.model_selection import train_test_split as tts 
from transformers import BertTokenizer, TFBertForSequenceClassification, AdamWeightDecay
import tensorflow as tf

x_train, x_test, y_train, y_test = tts(df['Comment'], df['Sentiment'], test_size=0.15, random_state=42)

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Function to tokenize the dataset properly
def tokenize_data(texts, labels, tokenizer, max_length=256):
    encodings = tokenizer(texts.tolist(), truncation=True, padding=True, max_length=max_length)
    dataset = tf.data.Dataset.from_tensor_slices((dict(encodings), labels))
    return dataset

# Tokenize the dataset
train_dataset = tokenize_data(x_train, y_train, tokenizer)
test_dataset = tokenize_data(x_test, y_test, tokenizer)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

# Compute class weights
class_weights = compute_class_weight(
    class_weight="balanced", classes=np.unique(y_train), y=y_train
)
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

In [ ]:
import tensorflow as tf
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# Load pre-trained BERT model for classification
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

# Define optimizer and loss function
optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)
loss_fn = SparseCategoricalCrossentropy(from_logits=True)

# Compile the model
model.compile(optimizer=optimizer, loss=loss_fn, metrics=["accuracy"])

In [ ]:
# Train the model with class weights and batch size of 1
model.fit(train_dataset.batch(4),  
          epochs=3, 
          class_weight=class_weights_dict)

In [ ]:
loss, accuracy = model.evaluate(test_dataset.batch(4))
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")